### TITLE : WEBSCRAPING TO EXTRACT  STOCK FUNDAMENTAL  DATA
<p> Author : BRYAN LIM YUQIANG </p> 
<p> Contact me : </p>
<li>Bryanlimyuqiang@gmail.com</li>
<li><a href = "https://www.linkedin.com/in/bryanlimyuqiang/">Linkedin </a></li> 

<b> instructions : </b><br>
<li> Do "pip install bs4 in your terminal before you start" </li>
<li> We will be scraping Yahoo Finance Financials </li>
<li> Will require some knowledge of HTML  </li>

In [3]:
import requests
from bs4 import BeautifulSoup

### BASIC WEBSCRAPPING

In [4]:
url = "https://sg.finance.yahoo.com/quote/TSLA/balance-sheet?p=TSLA"
page =requests.get(url) #200 would means success 
page_content = page.content
soup = BeautifulSoup(page_content, 'html.parser')
table = soup.find_all()

### Webscraping code off Yahoo Finance 

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

tickers = ["AMZN","TSLA"] #list of tickers whose financial data needs to be extracted
financial_dir = {}


for ticker in tickers:
    #getting balance sheet data from yahoo finance for the given ticker
    temp_dir = {}
    url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
    page = requests.get(url)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "rw-expnded"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting income statement data from yahoo finance for the given ticker
    url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
    page = requests.get(url)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "rw-expnded"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting cashflow statement data from yahoo finance for the given ticker
    url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
    page = requests.get(url)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "rw-expnded"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting key statistics data from yahoo finance for the given ticker
    url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
    page = requests.get(url)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c) "})
    for t in tabl:
        rows = t.find_all("tr")
        for row in rows:
            if len(row.get_text(separator='|').split("|")[0:2])>0:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
    
    #combining all extracted information with the corresponding ticker
    financial_dir[ticker] = temp_dir


#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
tickers = combined_financials.columns
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]

In [16]:
financial_dir

{'AMZN': {'Assets': 'Current assets',
  'Current assets': 'Cash',
  'Cash': 'Cash and cash equivalents',
  'Cash and cash equivalents': '3,60,92,000',
  'Other Short Term Investments': '1,89,29,000',
  'Total cash': '5,50,21,000',
  'Net receivables': '2,08,16,000',
  'Inventory': '-26,05,000',
  'Total current assets': '9,63,34,000',
  'Non-current assets': 'Property, plant and equipment',
  'Property, plant and equipment': 'Gross property, plant and equipment',
  'Gross property, plant and equipment': '14,48,21,000',
  'Accumulated depreciation': '-4,69,75,000',
  'Net property, plant and equipment': '9,78,46,000',
  'Goodwill': '1,47,54,000',
  'Intangible assets': '40,49,000',
  'Other long-term assets': '1,22,65,000',
  'Total non-current assets': '12,89,14,000',
  'Total assets': '22,52,48,000',
  "Liabilities and stockholders' equity": 'Liabilities',
  'Liabilities': 'Current liabilities',
  'Current liabilities': 'Accounts payable',
  'Accounts payable': '65,33,000',
  'Accrued

In [15]:
combined_financials

,AMZN,TSLA
Cash and cash equivalents,"3,60,92,000","62,68,000"
Other Short Term Investments,"1,89,29,000",NaN
Total cash,"5,50,21,000","62,68,000"
Net receivables,"2,08,16,000","13,24,000"
Inventory,"-26,05,000","-6,01,000"
...,...,...
Capital expenditure,"-2,03,66,000","-16,13,000"
Current debt,NaN,"17,85,000"
Other current liabilities,NaN,"5,77,000"
Other investing activities,NaN,-
